## **Gradient descent**

In [ ]:
alpha = 0.02
x = 10.0

def compute_grad(x):
    grad = 2 * x - 4
    return grad

for _ in range(1000):
    x = x - alpha * compute_grad(x)
print(x)

## **Gradient descent with Pytorch**

In [ ]:
import torch

In [ ]:
x = torch.tensor(10, dtype=torch.float, requires_grad=True)
cost = torch.sum(x * x - 4 * x)
cost.backward() # calculate gradient using "backward"
print(x.grad) # gradient

## **Gradient Descent for regression**

## **Gradient Descent for logistic regression**